##### Import relevant packages

In [3]:
import pandas as pd
import matplotlib

##### Reference similarweb .csv files and read to DataFrames (need to be more programmatic)

In [4]:
feb_data_file = 'similarweb_feb.csv'
mar_data_file = 'similarweb_march.csv'

In [5]:
df_feb = pd.read_csv(feb_data_file)
df_mar = pd.read_csv(mar_data_file)

##### Preview the top 5 rows of the February DataFrame

In [6]:
df_feb.head()

,Account Name,Name,Average Visit Duration,Website Category,2nd Traffic Country,Bounce Rate,Category Rank,Desktop Visits Share,Direct Visits Share,Display Ads Visits Share,...,Mobile Web Visits Share,Monthly Unique Visitors,Pages Per Visit,Paid Search Visits Share,Total Monthly Visits,Total Visits MoM Growth,Top Traffic Country,Social Visits Share,Account ID,Snapshot Date
0,Meez Culinary Solutions LLC,getmeez.com,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,...,0.00,0,NaN,NaN,NaN,-100.00,NaN,NaN,0012K00001fHHQN,2/1/20
1,PolyOne,polyone.com,0:04:07,Heavy Industry and Engineering/Chemical Industry,Canada,40.55,219.0,86.95,45.75,NaN,...,13.05,26475,4.45,2.48,64385.0,-18.36,United States,1.27,0012K00001ZKgZV,2/1/20
2,Town Boys and Girls Club,bgcnt.net,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,...,0.00,0,NaN,NaN,NaN,-100.00,NaN,NaN,0016A00000n9ySf,2/1/20
3,Key Packaging,keypackaging.com,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,...,0.00,0,NaN,NaN,NaN,-100.00,NaN,NaN,0016A00000n9vCp,2/1/20
4,Piedmont Rural Telephone Cooperative Inc,prtcnet.com,0:06:09,Health/Health,NaN,28.19,9777.0,90.92,86.11,NaN,...,9.08,8408,3.43,NaN,56630.0,-2.70,United States,0.04,0016A00000n9wkV,2/1/20


##### Reorder columns to most logical sequencing (not required)

In [7]:
df_feb.columns
columns_ordered = ['Account ID','Account Name', 'Name', 'Domain', 'Website Category','Category Rank',
                   'Global Rank', 'Top Traffic Country','2nd Traffic Country', 'Has Data','Total Monthly Visits',
                   'Monthly Unique Visitors','Bounce Rate', 'Pages Per Visit', 'Average Visit Duration',
                   'Total Visits MoM Growth','Desktop Visits Share', 'Mobile Web  Visits Share',
                   'Direct Visits Share','Display Ads Visits Share','Mail Visits Share',
                   'Paid Search Visits Share', 'Social Visits Share','Snapshot Date']
df_feb = df_feb[columns_ordered]
df_mar = df_mar[columns_ordered]

##### Fill all #NAs with 0s for Monthly Visits and Paid Search/Display Ads columns

In [8]:
df_feb['Total Monthly Visits'].fillna(0, inplace=True)
df_feb['Paid Search Visits Share'].fillna(0, inplace=True)
df_feb['Display Ads Visits Share'].fillna(0, inplace=True)

df_mar['Total Monthly Visits'].fillna(0, inplace=True)
df_mar['Paid Search Visits Share'].fillna(0, inplace=True)
df_mar['Display Ads Visits Share'].fillna(0, inplace=True)

##### Compute Total Ad Spend Visits Share (Paid Search + Display Ads) then calculate # of visits from Total Monthly Visits

In [9]:
df_feb['Total Ad Spend Visits Share'] = df_feb['Paid Search Visits Share'] + df_feb['Display Ads Visits Share']
df_feb['Total Ad Spend Visits'] = df_feb['Total Monthly Visits']*df_feb['Total Ad Spend Visits Share']/100

df_mar['Total Ad Spend Visits Share'] = df_mar['Paid Search Visits Share'] + df_mar['Display Ads Visits Share']
df_mar['Total Ad Spend Visits'] = df_mar['Total Monthly Visits']*df_mar['Total Ad Spend Visits Share']/100

###### Slim down DataFrame for only relevant columns to be output to .csv

In [10]:
df_feb_slim = df_feb[['Account ID', 'Account Name', 'Domain', 'Website Category', 'Total Monthly Visits',
                     'Total Visits MoM Growth','Total Ad Spend Visits Share','Total Ad Spend Visits']]

df_mar_slim = df_mar[['Account ID', 'Account Name', 'Domain', 'Website Category', 'Total Monthly Visits',
                     'Total Visits MoM Growth','Total Ad Spend Visits Share','Total Ad Spend Visits']]

###### Left join March and February, add suffixes to column names and drop redundant columns

In [11]:
merged = df_mar_slim.merge(df_feb_slim,on=['Account ID'],how='left',suffixes=('_march', '_feb'))
merged.drop(columns=['Account Name_feb', 'Domain_feb', 'Website Category_feb'], axis=1, inplace=True)

##### Export merged DataFrame to .csv file on local repository

In [12]:
merged.to_csv('feb_march_similarweb.csv')

##### Other scratchwork below...

In [13]:
merged.columns

Index(['Account ID', 'Account Name_march', 'Domain_march',
       'Website Category_march', 'Total Monthly Visits_march',
       'Total Visits MoM Growth_march', 'Total Ad Spend Visits Share_march',
       'Total Ad Spend Visits_march', 'Total Monthly Visits_feb',
       'Total Visits MoM Growth_feb', 'Total Ad Spend Visits Share_feb',
       'Total Ad Spend Visits_feb'],
      dtype='object')

In [16]:
merged['Total Visits Delta'] = merged['Total Monthly Visits_march'] - merged['Total Monthly Visits_feb']
merged['Total Ad Visits Delta'] = merged['Total Ad Spend Visits_march'] - merged['Total Ad Spend Visits_feb']

In [19]:
merged[['Website Vertical','Website Industry']] = merged['Website Category_march'].str.split('/',expand=True)

In [22]:
vertical = merged.groupby('Website Vertical')[['Total Visits Delta','Total Ad Visits Delta']].apply(sum)

In [25]:
vertical.head(50)

,Total Visits Delta,Total Ad Visits Delta
Website Vertical,,
Adult,4.094991e+06,1.138613e+06
Arts and Entertainment,3.266995e+10,3.483256e+07
Business and Consumer Services,1.138902e+08,1.306918e+07
Community and Society,3.239878e+07,-8.209715e+05
Computers Electronics and Technology,1.646440e+10,2.284277e+07
E-commerce and Shopping,4.624178e+08,-6.103270e+07
Finance,3.673970e+08,-1.453591e+06
Food and Drink,2.178794e+08,-2.778691e+06
Gambling,-1.201752e+07,7.111972e+05
